<a href="https://colab.research.google.com/github/pullz6/Data_Pipeline-TFX/blob/main/Pipeline.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
! pip3 install --upgrade pip

In [2]:
! pip install tfx

Importing the dataset

In [3]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


Extracing the dataset from the zipped location

In [ ]:
import zipfile
zip_ref = zipfile.ZipFile("/content/drive/My Drive/sample_data/dataset.zip",'r')
zip_ref.extractall("/content/drive/My Drive/sample_data/movie_reviews")
zip_ref.close()

Divide the dataset into training and validation

In [4]:
from tfx.proto import example_gen_pb2

output = example_gen_pb2.Output(
    split_config = example_gen_pb2.SplitConfig(
      splits = [
          example_gen_pb2.SplitConfig.Split(name='train',hash_buckets=8),
          example_gen_pb2.SplitConfig.Split(name='eval',hash_buckets=2)
          ])
)

In [5]:
from tfx.components import CsvExampleGen

example_gen = CsvExampleGen(input_base='/content/drive/My Drive/sample_data/movie_reviews/2_reviews_per_movie_raw', output_config=output)

In [6]:
from tfx.orchestration.experimental.interactive.interactive_context import InteractiveContext

context = InteractiveContext(pipeline_root = '/content/drive/My Drive/sample_data/movie_reviews/Pipeline')

In [ ]:
context.run(example_gen)

Reading our transformed file

In [ ]:
import tensorflow as tf
import os
import pprint
pp = pprint

train_uri = os.path.join(example_gen.outputs['examples'].get()[0].uri, 'Split-train')

tfrecord_filenames = [os.path.join(train_uri, name) for name in os.listdir(train_uri)]

dataset = tf.data.TFRecordDataset(tfrecord_filenames, compression_type='GZIP')

for tfrecord in dataset.take(2):
    serialized_example = tfrecord.numpy()
    example = tf.train.Example()
    example.ParseFromString(serialized_example)
    pp.pprint(example)

features {
  feature {
    key: "\"date\""
    value {
      bytes_list {
        value: "30 January 2019"
      }
    }
  }
  feature {
    key: "\"helpful\""
    value {
      int64_list {
        value: 102
      }
    }
  }
  feature {
    key: "\"rating\""
    value {
      bytes_list {
        value: "7"
      }
    }
  }
  feature {
    key: "\"review\""
    value {
      bytes_list {
        value: "This movie is full of suspense. It makes you guess about what is real and what is not. It happens more than once that you have to wonder about what is the truth and who is lying.<br/><br/>Because you are just as clueless as the main character, Michelle, you really get to experience the same type of emotions and confusion as she is. This not only makes you feel closer to Michelle as a character, but also to the overall story.<br/><br/>They did, however, ruin this connection that they build over the course of the story by making this sloppy, rushed and just strange ending. If they had

Data Validation

In [ ]:
from tfx.components import StatisticsGen

statistics_gen = StatisticsGen(
    examples = example_gen.outputs['examples']
)

context.run(statistics_gen)

ExecutionResult(
    component_id: StatisticsGen
    execution_id: 2
    outputs:
        statistics: OutputChannel(artifact_type=ExampleStatistics, producer_component_id=StatisticsGen, output_key=statistics, additional_properties={}, additional_custom_properties={}, _input_trigger=None, _is_async=False))

Visualizing Statistics

In [ ]:
context.show(statistics_gen.outputs['statistics'])

Data Cleaning with Dask

In [ ]:
! pip3 install --upgrade dask

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.2/1.2 MB 19.0 MB/s eta 0:00:00
  Attempting uninstall: cloudpickle
    Found existing installation: cloudpickle 2.2.1
    Uninstalling cloudpickle-2.2.1:
      Successfully uninstalled cloudpickle-2.2.1
  Attempting uninstall: dask
    Found existing installation: dask 2024.7.1
    Uninstalling dask-2024.7.1:
      Successfully uninstalled dask-2024.7.1
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
apache-beam 2.58.1 requires cloudpickle~=2.2.1, but you have cloudpickle 3.0.0 which is incompatible.
dask-expr 1.1.9 requires dask==2024.7.1, but you have dask 2024.8.1 which is incompatible.
distributed 2024.7.1 requires dask==2024.7.1, but you have dask 2024.8.1 which is incompatible.
tensorflow-data-validation 1.15.1 requires pandas<2,>=1.0, but you have pandas 2.2.2 which is incompatible.
tensorflow-model-analysis 

In [ ]:
! pip install "dask[dataframe]"

In [ ]:
import dask.dataframe as dd

df1 = dd.read_csv('/content/drive/My Drive/sample_data/movie_reviews/2_reviews_per_movie_raw/*.csv')

df1.head()

ImportError: Dask dataframe requirements are not installed.

Please either conda or pip install as follows:

  conda install dask                     # either conda install
  python -m pip install "dask[dataframe]" --upgrade  # or python -m pip install